In [1]:
# Model Training
from datasets import load_from_disk

datasets = load_from_disk("./encoded_data4")
datasets

c:\Users\user\Desktop\QuestionQuestionModel\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Spreadsheets and Databases', 'Threats and Security on the Internet', '__index_level_0__'],
        num_rows: 644
    })
    valid: Dataset({
        features: ['String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Spreadsheets and Databases', '

In [2]:
datasets = datasets.remove_columns(['__index_level_0__'])

In [3]:
datasets

DatasetDict({
    train: Dataset({
        features: ['String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Spreadsheets and Databases', 'Threats and Security on the Internet'],
        num_rows: 644
    })
    valid: Dataset({
        features: ['String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Spreadsheets and Databases', 'Threats and Security 

In [4]:
model_labels = [label for label in datasets['train'].features.keys() if label not in ['Unnamed: 0', 'String', '__index_level_0__']]

In [5]:
model_labels

['Algorithm Design',
 'Basic Machine Organisation',
 'Computer System',
 'Data Manipulation and Analysis',
 'Data Organisation and Data Control',
 'Elementary Web Authoring',
 'Health and Ethical Issues',
 'Information Processing',
 'Intellectual Property',
 'Internet Services and Applications',
 'Multimedia Elements',
 'Networking and Internet Basics',
 'Program Development',
 'Spreadsheets and Databases',
 'Threats and Security on the Internet']

In [6]:
datasets["train"].column_names

['String',
 'Algorithm Design',
 'Basic Machine Organisation',
 'Computer System',
 'Data Manipulation and Analysis',
 'Data Organisation and Data Control',
 'Elementary Web Authoring',
 'Health and Ethical Issues',
 'Information Processing',
 'Intellectual Property',
 'Internet Services and Applications',
 'Multimedia Elements',
 'Networking and Internet Basics',
 'Program Development',
 'Spreadsheets and Databases',
 'Threats and Security on the Internet']

In [7]:
datasets["train"].features

{'String': Value(dtype='string', id=None),
 'Algorithm Design': Value(dtype='int32', id=None),
 'Basic Machine Organisation': Value(dtype='int32', id=None),
 'Computer System': Value(dtype='int32', id=None),
 'Data Manipulation and Analysis': Value(dtype='int32', id=None),
 'Data Organisation and Data Control': Value(dtype='int32', id=None),
 'Elementary Web Authoring': Value(dtype='int32', id=None),
 'Health and Ethical Issues': Value(dtype='int32', id=None),
 'Information Processing': Value(dtype='int32', id=None),
 'Intellectual Property': Value(dtype='int32', id=None),
 'Internet Services and Applications': Value(dtype='int32', id=None),
 'Multimedia Elements': Value(dtype='int32', id=None),
 'Networking and Internet Basics': Value(dtype='int32', id=None),
 'Program Development': Value(dtype='int32', id=None),
 'Spreadsheets and Databases': Value(dtype='int32', id=None),
 'Threats and Security on the Internet': Value(dtype='int32', id=None)}

In [8]:
#print(f"{datasets.train.column_names}\n{datasets.train.features}")

In [9]:
model_labels

['Algorithm Design',
 'Basic Machine Organisation',
 'Computer System',
 'Data Manipulation and Analysis',
 'Data Organisation and Data Control',
 'Elementary Web Authoring',
 'Health and Ethical Issues',
 'Information Processing',
 'Intellectual Property',
 'Internet Services and Applications',
 'Multimedia Elements',
 'Networking and Internet Basics',
 'Program Development',
 'Spreadsheets and Databases',
 'Threats and Security on the Internet']

In [10]:
id2label = {idx:label for idx, label in enumerate(model_labels)}
label2id = {label:idx for idx, label in enumerate(model_labels)}

In [11]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import numpy as np

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [12]:
def encode_data(dataset):
    text = dataset["String"]
    # tokenize string
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=70)
    # create encoded array list with labels
    labelsBatch = {x: dataset[x] for x in dataset.keys() if x in model_labels}
    labels_matrix = np.zeros((len(text), len(model_labels)))
    for idx, label in enumerate(model_labels):
        labels_matrix[:, idx] = labelsBatch[label]
    
    encoding["labels"] = labels_matrix.tolist()
    return encoding

In [13]:
# tokenize the dataset
encoded_ds = datasets.map(encode_data, batched=True, remove_columns=datasets['train'].column_names)

Loading cached processed dataset at c:\Users\user\Desktop\QuestionQuestionModel\model\encoded_data4\train\cache-7700797c4d7f932e.arrow
Loading cached processed dataset at c:\Users\user\Desktop\QuestionQuestionModel\model\encoded_data4\valid\cache-ea223c76f3455998.arrow
Loading cached processed dataset at c:\Users\user\Desktop\QuestionQuestionModel\model\encoded_data4\test\cache-459aacbf91ebbdc4.arrow


In [14]:
encoded_ds['train']['labels'][0]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]

In [15]:
[id2label[idx] for idx, label in enumerate(encoded_ds['train'][15]['labels']) if label == 1.0]

['Data Organisation and Data Control']

In [16]:
import torch
from torch import cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

encoded_ds.set_format("torch")

Using device: cuda



In [17]:
encoded_ds["train"]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 644
})

In [18]:

# define model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", problem_type="multi_label_classification", num_labels=len(model_labels), id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

In [19]:
# train model
batch_size = 8
metric_name = "f1"

In [20]:
import evaluate
metric  = evaluate.load("accuracy")

In [21]:
import time

train_name_dir = time.strftime(f"%Y%m%d_%H%M%S$-train-model-{model.name_or_path}")
train_name_dir

'20230329_103603$-train-model-distilbert-base-uncased'

In [22]:
from transformers import TrainingArguments, Trainer


args = TrainingArguments(
    output_dir="./distlBert-v10/"+ train_name_dir,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs= 81,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='./distlBert-v10/logging/'+ train_name_dir
)

In [23]:
args.learning_rate

2e-05

In [24]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)

c:\Users\user\Desktop\QuestionQuestionModel\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, balanced_accuracy_score, hamming_loss
from transformers import EvalPrediction


def multi_label_metrics(predictions, labels, threshold=0.50):
    # apply sigmoid on predictions fitting (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    balanced_accuracy = balanced_accuracy_score(y_true.argmax(axis=1), y_pred.argmax(axis=1))
    # logits, labels = predictions
    # prediction = np.argmax(logits, axis=-1)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'balanced_accuracy': balanced_accuracy,
               }
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

def compute_metrics_new(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [26]:
encoded_ds['train'][0]['labels'].type()

'torch.FloatTensor'

In [27]:
# model.to(device)

In [28]:
outputs = model(input_ids=encoded_ds['train']['input_ids'][0].unsqueeze(0), labels=encoded_ds['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.6868, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.0290,  0.0643, -0.0415,  0.1615, -0.1406,  0.1397, -0.0361, -0.0566,
         -0.1169, -0.0155,  0.0683, -0.0058,  0.0523, -0.0006, -0.1242]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [29]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_ds["train"],
    eval_dataset=encoded_ds["valid"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [30]:
trainer.train()

***** Running training *****
  Num examples = 644
  Num Epochs = 81
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6561
  Number of trainable parameters = 66965007
  1%|          | 81/6561 [00:10<05:08, 20.98it/s]  ***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
                                                 
  1%|          | 81/6561 [00:11<05:08, 20.98it/s]Saving model checkpoint to ./distlBert-v10/20230329_103603$-train-model-distilbert-base-uncased\checkpoint-81
Configuration saved in ./distlBert-v10/20230329_103603$-train-model-distilbert-base-uncased\checkpoint-81\config.json


{'eval_loss': 0.2775842249393463, 'eval_f1': 0.0, 'eval_roc_auc': 0.5, 'eval_accuracy': 0.0, 'eval_balanced_accuracy': 0.06666666666666667, 'eval_runtime': 0.1493, 'eval_samples_per_second': 542.619, 'eval_steps_per_second': 73.689, 'epoch': 1.0}


Model weights saved in ./distlBert-v10/20230329_103603$-train-model-distilbert-base-uncased\checkpoint-81\pytorch_model.bin
tokenizer config file saved in ./distlBert-v10/20230329_103603$-train-model-distilbert-base-uncased\checkpoint-81\tokenizer_config.json
Special tokens file saved in ./distlBert-v10/20230329_103603$-train-model-distilbert-base-uncased\checkpoint-81\special_tokens_map.json
  2%|▏         | 161/6561 [00:15<05:05, 20.98it/s]***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
                                                  
  2%|▏         | 162/6561 [00:16<05:05, 20.98it/s]Saving model checkpoint to ./distlBert-v10/20230329_103603$-train-model-distilbert-base-uncased\checkpoint-162
Configuration saved in ./distlBert-v10/20230329_103603$-train-model-distilbert-base-uncased\checkpoint-162\config.json


{'eval_loss': 0.2513124346733093, 'eval_f1': 0.0, 'eval_roc_auc': 0.5, 'eval_accuracy': 0.0, 'eval_balanced_accuracy': 0.06666666666666667, 'eval_runtime': 0.1318, 'eval_samples_per_second': 614.462, 'eval_steps_per_second': 83.445, 'epoch': 2.0}


Model weights saved in ./distlBert-v10/20230329_103603$-train-model-distilbert-base-uncased\checkpoint-162\pytorch_model.bin
tokenizer config file saved in ./distlBert-v10/20230329_103603$-train-model-distilbert-base-uncased\checkpoint-162\tokenizer_config.json
Special tokens file saved in ./distlBert-v10/20230329_103603$-train-model-distilbert-base-uncased\checkpoint-162\special_tokens_map.json
  4%|▎         | 242/6561 [00:20<05:02, 20.88it/s]***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
                                                  
  4%|▎         | 243/6561 [00:21<05:02, 20.88it/s]Saving model checkpoint to ./distlBert-v10/20230329_103603$-train-model-distilbert-base-uncased\checkpoint-243
Configuration saved in ./distlBert-v10/20230329_103603$-train-model-distilbert-base-uncased\checkpoint-243\config.json


...

100%|█████████▉| 6560/6561 [15:33<00:00, 20.36it/s]***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
                                                   
100%|██████████| 6561/6561 [15:33<00:00, 20.36it/s]Saving model checkpoint to ./distlBert-v10/20230329_103603$-train-model-distilbert-base-uncased\checkpoint-6561
Configuration saved in ./distlBert-v10/20230329_103603$-train-model-distilbert-base-uncased\checkpoint-6561\config.json


{'eval_loss': 0.11819165199995041, 'eval_f1': 0.8263473053892216, 'eval_roc_auc': 0.8958483531423157, 'eval_accuracy': 0.7901234567901234, 'eval_balanced_accuracy': 0.8210533910533911, 'eval_runtime': 0.1314, 'eval_samples_per_second': 616.21, 'eval_steps_per_second': 83.683, 'epoch': 81.0}


Model weights saved in ./distlBert-v10/20230329_103603$-train-model-distilbert-base-uncased\checkpoint-6561\pytorch_model.bin
tokenizer config file saved in ./distlBert-v10/20230329_103603$-train-model-distilbert-base-uncased\checkpoint-6561\tokenizer_config.json
Special tokens file saved in ./distlBert-v10/20230329_103603$-train-model-distilbert-base-uncased\checkpoint-6561\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./distlBert-v10/20230329_103603$-train-model-distilbert-base-uncased\checkpoint-4455 (score: 0.8466257668711655).
100%|██████████| 6561/6561 [15:43<00:00,  6.96it/s]

{'train_runtime': 943.5738, 'train_samples_per_second': 55.283, 'train_steps_per_second': 6.953, 'train_loss': 0.023326866827522486, 'epoch': 81.0}


TrainOutput(global_step=6561, training_loss=0.023326866827522486, metrics={'train_runtime': 943.5738, 'train_samples_per_second': 55.283, 'train_steps_per_second': 6.953, 'train_loss': 0.023326866827522486, 'epoch': 81.0})

In [31]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
100%|██████████| 11/11 [00:00<00:00, 61.29it/s]


{'eval_loss': 0.1109214723110199,
 'eval_f1': 0.8466257668711655,
 'eval_roc_auc': 0.8976198323274354,
 'eval_accuracy': 0.7901234567901234,
 'eval_balanced_accuracy': 0.8210533910533911,
 'eval_runtime': 0.2446,
 'eval_samples_per_second': 331.163,
 'eval_steps_per_second': 44.973,
 'epoch': 81.0}

In [32]:
prediction_result = trainer.predict(encoded_ds['test'])

***** Running Prediction *****
  Num examples = 81
  Batch size = 8
100%|██████████| 11/11 [00:00<00:00, 95.76it/s]


In [33]:
print(prediction_result.metrics)

{'test_loss': 0.15585118532180786, 'test_f1': 0.7701863354037267, 'test_roc_auc': 0.8664268380944272, 'test_accuracy': 0.7283950617283951, 'test_balanced_accuracy': 0.779071669071669, 'test_runtime': 0.1858, 'test_samples_per_second': 435.929, 'test_steps_per_second': 59.2}


In [34]:
trainer.save_model("./distlBert-model-v10")

Saving model checkpoint to ./distlBert-model-v10
Configuration saved in ./distlBert-model-v10\config.json
Model weights saved in ./distlBert-model-v10\pytorch_model.bin
tokenizer config file saved in ./distlBert-model-v10\tokenizer_config.json
Special tokens file saved in ./distlBert-model-v10\special_tokens_map.json


In [35]:
tokenizer.save_pretrained("./distlBert-model-v10/tokenizer")

tokenizer config file saved in ./distlBert-model-v10/tokenizer\tokenizer_config.json
Special tokens file saved in ./distlBert-model-v10/tokenizer\special_tokens_map.json


('./distlBert-model-v10/tokenizer\\tokenizer_config.json',
 './distlBert-model-v10/tokenizer\\special_tokens_map.json',
 './distlBert-model-v10/tokenizer\\vocab.txt',
 './distlBert-model-v10/tokenizer\\added_tokens.json')